In [1]:
import freesasa
from Bio.PDB import PDBParser
from pdb_utils import *


In [2]:
cry = './4gv1_protein.pdb'
af = './4gv1_alpha.pdb'
ligand = './4gv1_ori.sdf'

cry_residues = pocket_trunction(cry, ligand, save_name='./cry_pocket.pdb')

In [3]:
parser = PDBParser()
af_structure = parser.get_structure("af", af)
af_residues = extract_alphafold_pocket(cry_residues, af_structure)

/home/haotian/software/miniconda3/envs/carbon/lib/python3.9/site-packages/Bio/PDB/PDBParser.py:395: PDBConstructionWarning: Ignoring unrecognized record 'END' at line 3921
  warnings.warn(


In [4]:
residues_saver(af_structure, af_residues, './af_pocket.pdb')

In [5]:

cry_pdb_file = './cry_pocket.pdb'
af_pdb_file = './af_pocket.pdb'

crystal_volume = calculate_pocket_sasa(cry_pdb_file)
alphafpld_volume = calculate_pocket_sasa(af_pdb_file)

print(f"Crystall Pocket 1 volume: {crystal_volume}")
print(f"AlphaFold Pocket 2 volume: {alphafpld_volume}")

Crystall Pocket 1 volume: 5842.138363032393
AlphaFold Pocket 2 volume: 6014.777247963261


In [36]:
from glob import glob
pkt_files = np.sort(glob('./af_pocket_out/pockets/*.pdb'))

distance = np.zeros(len(pkt_files))
pkts_info = []
lig_center = read_ligand_coords(ligand_sdf).mean(axis=0)
for idx,pkt_file in enumerate(pkt_files):
    pkt_info = read_pocket_info(pkt_file)
    pkts_info.append(pkt_info)
    distance[idx] = np.linalg.norm(lig_center - pkt_info[0])
real_pocket_id = np.argmin(distance)
real_mc_volume = pkts_info[real_pocket_id][1]
real_hull_volume = pkts_info[real_pocket_id][2]

print('Monte Carlo Volume: ', real_mc_volume)
print('Convex Hull Volumes: ', real_hull_volume)

Monte Carlo Volume:  882.1127
Convex Hull Volumes:  351.6829


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from scipy.spatial import ConvexHull


def compute_convex_hull_volume(coords):
    tri = Delaunay(coords)
    tetrahedra = coords[tri.simplices]

    volume = 0.0
    for tetrahedron in tetrahedra:
        volume += np.abs(np.linalg.det(tetrahedron[:-1] - tetrahedron[-1])) / 6

    return volume

def plot_convex_hull(coords1, coords2, volume1, volume2):
    hull1 = ConvexHull(coords1)
    hull2 = ConvexHull(coords2)

    fig = plt.figure(figsize=(12, 6))
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')

    for ax, hull, volume, title in zip([ax1, ax2], [hull1, hull2], [volume1, volume2], ['Pocket 1', 'Pocket 2']):
        for simplex in hull.simplices:
            poly = Poly3DCollection([coords1[simplex]], alpha=0.5)
            poly.set_facecolor('blue')
            ax.add_collection3d(poly)

        ax.set_title(f'{title} - Volume: {volume:.2f}')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        ax.set_zlabel('Z')
        ax.set_xlim([-50, 50])
        ax.set_ylim([-50, 50])
        ax.set_zlim([-50, 50])
        ax.view_init(elev=20, azim=-35)

    plt.show()

def get_pdb_coords(pdbfile):
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('pocket', pdbfile)
    coords = np.array([atom.get_coord() for atom in structure.get_atoms()])
    return coords

coords1 = np.random.rand(30, 3) * 100 - 50  # Generate random coordinates for pocket 1
coords2 = np.random.rand(30, 3) * 100 - 50  # Generate random coordinates for pocket 2
coords1 = get_pdb_coords(crystal)
coords2 = get_pdb_coords(af)
volume1 = compute_convex_hull_volume(coords1)  # Replace with your function to compute the volume
volume2 = compute_convex_hull_volume(coords2)  # Replace with your function to compute the volume

plot_convex_hull(coords1, coords2, volume1, volume2)
